In [1]:
#Not working

In [2]:
import os
import sys
import time
import multiprocessing
import numpy as np
import pandas as pd
import tensorflow as tf
from nets import densenet  # Download from https://github.com/pudae/tensorflow-densenet
from tensorflow.python.framework import dtypes
from tensorflow.python.framework.ops import convert_to_tensor
from tensorflow.contrib.data import Iterator
from tensorflow.contrib.slim.python.slim.nets import resnet_v1
from PIL import Image
import random
from common.utils import download_data_chextxray, get_imgloc_labels, get_train_valid_test_split
from common.utils import compute_roc_auc
slim = tf.contrib.slim

In [3]:
tf.__version__

'1.4.0'

In [4]:
CPU_COUNT = multiprocessing.cpu_count()
print("CPUs: ", CPU_COUNT)

CPUs:  12


In [5]:
# Globals
CLASSES = 14
WIDTH = 224
HEIGHT = 224
CHANNELS = 3
LR = 0.0001  # Effective learning-rate will decrease as BATCHSIZE rises
EPOCHS = 5
BATCHSIZE = 64  # Chainer auto scales batch
IMAGENET_RGB_MEAN = np.array([123.68, 116.78, 103.94], dtype=np.float32)
IMAGENET_SCALE_FACTOR = 0.017
TOT_PATIENT_NUMBER = 30805  # From data

In [6]:
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)
# Model checkpoint
#CHKPOINT = 'tfdensenet/tf-densenet121.ckpt'
CHKPOINT = 'resnet_v1_50.ckpt'

chestxray/images chestxray/Data_Entry_2017.csv


In [7]:
%%time
# Download data
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 641 ms, sys: 228 ms, total: 870 ms
Wall time: 869 ms


In [8]:
#####################################################################################################
## Data Loading

In [9]:
class XrayData():
    
    def __init__(self, img_dir, lbl_file, patient_ids, mode='inference', 
                 width=WIDTH, height=HEIGHT, batch_size=BATCHSIZE, 
                 imagenet_mean=IMAGENET_RGB_MEAN, imagenet_scaling = IMAGENET_SCALE_FACTOR):
        # Get data
        self.img_locs, self.labels = get_imgloc_labels(img_dir, lbl_file, patient_ids)
        self.data_size = len(self.labels)
        self.imagenet_mean = imagenet_mean
        self.imagenet_scaling = imagenet_scaling
        self.width = width
        self.height = height
        # Create dataset
        # Performance: https://www.tensorflow.org/versions/master/performance/datasets_performance
        # Following: https://stackoverflow.com/a/48096625/6772173
        data = tf.data.Dataset.from_tensor_slices((self.img_locs, self.labels))
        # Processing
        if mode == 'training':
            print("Training")
            data = data.shuffle(self.data_size).repeat().map(self._parse_function_train,
                            num_parallel_calls=CPU_COUNT).prefetch(10*batch_size).batch(batch_size)
        else:
            print("Inference")
            data = data.map(self._parse_function_inference,
                            num_parallel_calls=CPU_COUNT).prefetch(10*batch_size).batch(batch_size)
        
        self.data = data        
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
        
        
    def _parse_function_train(self, filename, label):
        img_rgb, label = self._preprocess_image_labels(filename, label)
        # Super high CPU usuage bottlenecking GPU
        # Random crop
        img_rgb = tf.image.resize_images(img_rgb, [self.height+40, self.width+40])
        img_rgb = tf.random_crop(img_rgb, [self.height, self.width, 3])
        # Random flip
        img_rgb = tf.image.random_flip_left_right(img_rgb)
        return img_rgb, label
        
        
    def _parse_function_inference(self, filename, label):
        img_rgb, label = self._preprocess_image_labels(filename, label)
        # Resize to final dimensions
        img_rgb = tf.image.resize_images(img_rgb, [self.height, self.width])
        return img_rgb, label 
       
    
    def _preprocess_image_labels(self, filename, label):
        # load and preprocess the image
        img_decoded = tf.to_float(tf.image.decode_png(tf.read_file(filename), channels=3))
        img_centered = tf.subtract(img_decoded, self.imagenet_mean)
        img_rgb = img_centered * self.imagenet_scaling
        return img_rgb, tf.cast(label, dtype=tf.float32)

In [10]:
train_set, valid_set, test_set = get_train_valid_test_split(TOT_PATIENT_NUMBER, shuffle=True)

train:21563 valid:3080 test:6162


In [11]:
with tf.device('/cpu:0'):
    # Create dataset for iterator
    train_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=train_set,  
                             mode='training')
    valid_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=valid_set)
    test_dataset  = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=test_set)

Training
Loaded 78899 labels and 78899 images
Inference
Loaded 11080 labels and 11080 images
Inference
Loaded 22141 labels and 22141 images


In [12]:
#####################################################################################################
## Helper Functions

In [13]:
def get_symbol(model_name, in_tensor, is_training, chkpoint, out_features):
    if model_name == 'resnet50':
        # Import symbol
        with slim.arg_scope(resnet_v1.resnet_arg_scope()):
            base_model, _ = resnet_v1.resnet_v1_50(in_tensor, 
                                                   num_classes=None, 
                                                   is_training=is_training)
        # Attach extra layers
        fc = tf.layers.dense(base_model, out_features, name='output')
        # Activation function will be included in loss
        sym = tf.reshape(fc, shape=[-1, out_features])
    elif model_name == 'densenet121':
        pass
    else:
        raise ValueError("Unknown model-name")
    return sym

In [14]:
def model_fn(features, labels, mode, params):
    # Create symbol
    # is_training=True?
    # https://github.com/tensorflow/models/issues/3556
    sym = get_symbol(model_name='resnet50',
                     in_tensor=features, 
                     #is_training=(mode == tf.estimator.ModeKeys.TRAIN),
                     is_training=True,
                     chkpoint=params["checkpoint"],
                     out_features=params["n_classes"])
    # Predictions
    predictions = tf.sigmoid(sym)   
    # ModeKeys.PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions)
    # Optimizer & Loss
    optimizer = tf.train.AdamOptimizer(params['lr'], beta1=0.9, beta2=0.999)
    loss_fn = tf.losses.sigmoid_cross_entropy(labels, sym)
    loss = tf.reduce_mean(loss_fn)
    if mode == tf.estimator.ModeKeys.TRAIN:
        # Restore model from chkpoint (only when training)
        variables_to_restore = slim.get_variables_to_restore(exclude=['output'])
        #https://stackoverflow.com/questions/47867748/transfer-learning-with-tf-estimator-estimator-framework
        tf.train.init_from_checkpoint(params['checkpoint'], 
                              {v.name.split(':')[0]: v for v in variables_to_restore})
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))

In [15]:
def train_input_fn():
    return train_dataset.data.make_one_shot_iterator().get_next()
def valid_input_fn():
    return valid_dataset.data.make_one_shot_iterator().get_next()
def test_input_fn():
    return test_dataset.data.make_one_shot_iterator().get_next()

In [16]:
# Create Estimator
nn = tf.estimator.Estimator(model_fn=model_fn,
                            params={"lr":LR, 
                                    "checkpoint":CHKPOINT,
                                    "n_classes":CLASSES})

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_master': '', '_num_worker_replicas': 1, '_is_chief': True, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_task_id': 0, '_session_config': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_task_type': 'worker', '_save_summary_steps': 100, '_model_dir': '/tmp/tmpjwebku45', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc3cdb62550>}


In [17]:
%%time
# 48mins
nn.train(train_input_fn, 
         max_steps=EPOCHS*(train_dataset.data_size//BATCHSIZE))

INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_4/bottleneck_v1/conv3/BatchNorm/moving_mean:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_4/bottleneck_v1/conv3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable resnet_v1_50/block4/unit_3/bottleneck_v1/conv3/BatchNorm/moving_mean:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block4/unit_3/bottleneck_v1/conv3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable resnet_v1_50/block1/unit_2/bottleneck_v1/conv1/BatchNorm/moving_mean:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block1/unit_2/bottleneck_v1/conv1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_5/bottleneck_v1/conv1/BatchNorm/beta:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_5/bottleneck_v1/conv1/BatchNorm/beta
INFO:tensorflow:Initialize variable resnet_v1_50/block1/unit_2/bottleneck_v1/conv1/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block1/un

INFO:tensorflow:Initialize variable resnet_v1_50/block1/unit_3/bottleneck_v1/conv3/BatchNorm/gamma:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block1/unit_3/bottleneck_v1/conv3/BatchNorm/gamma
INFO:tensorflow:Initialize variable resnet_v1_50/block4/unit_3/bottleneck_v1/conv2/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block4/unit_3/bottleneck_v1/conv2/weights
INFO:tensorflow:Initialize variable resnet_v1_50/block4/unit_1/bottleneck_v1/conv1/BatchNorm/gamma:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block4/unit_1/bottleneck_v1/conv1/BatchNorm/gamma
INFO:tensorflow:Initialize variable resnet_v1_50/block4/unit_2/bottleneck_v1/conv1/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block4/unit_2/bottleneck_v1/conv1/weights
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_4/bottleneck_v1/conv3/BatchNorm/beta:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_4/bottleneck_v1/conv3/BatchNorm/beta
INFO:tensorflow:Ini

INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_6/bottleneck_v1/conv1/BatchNorm/moving_variance:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_6/bottleneck_v1/conv1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_2/bottleneck_v1/conv2/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/unit_2/bottleneck_v1/conv2/weights
INFO:tensorflow:Initialize variable resnet_v1_50/block4/unit_2/bottleneck_v1/conv3/BatchNorm/beta:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block4/unit_2/bottleneck_v1/conv3/BatchNorm/beta
INFO:tensorflow:Initialize variable resnet_v1_50/block4/unit_1/bottleneck_v1/shortcut/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block4/unit_1/bottleneck_v1/shortcut/weights
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_1/bottleneck_v1/conv1/BatchNorm/beta:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_1/bottleneck_v1/conv1/BatchNorm/

INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_2/bottleneck_v1/conv1/BatchNorm/gamma:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_2/bottleneck_v1/conv1/BatchNorm/gamma
INFO:tensorflow:Initialize variable resnet_v1_50/block1/unit_2/bottleneck_v1/conv2/BatchNorm/gamma:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block1/unit_2/bottleneck_v1/conv2/BatchNorm/gamma
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_4/bottleneck_v1/conv3/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/unit_4/bottleneck_v1/conv3/weights
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_3/bottleneck_v1/conv1/BatchNorm/gamma:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/unit_3/bottleneck_v1/conv1/BatchNorm/gamma
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_4/bottleneck_v1/conv2/BatchNorm/moving_variance:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_4/bottleneck_v1/conv2/BatchNo

INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_1/bottleneck_v1/conv3/BatchNorm/moving_mean:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_1/bottleneck_v1/conv3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_5/bottleneck_v1/conv3/BatchNorm/gamma
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_1/bottleneck_v1/conv3/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/unit_1/bottleneck_v1/conv3/weights
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_6/bottleneck_v1/conv1/BatchNorm/gamma:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_6/bottleneck_v1/conv1/BatchNorm/gamma
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_1/bottleneck_v1/shortcut/BatchNorm/beta:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/unit_1/bottleneck_v1/shortcut/

INFO:tensorflow:Initialize variable resnet_v1_50/block4/unit_3/bottleneck_v1/conv1/BatchNorm/moving_variance:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block4/unit_3/bottleneck_v1/conv1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable resnet_v1_50/block4/unit_3/bottleneck_v1/conv3/BatchNorm/moving_variance:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block4/unit_3/bottleneck_v1/conv3/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_3/bottleneck_v1/conv2/BatchNorm/moving_variance:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/unit_3/bottleneck_v1/conv2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_3/bottleneck_v1/conv2/BatchNorm/beta:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_3/bottleneck_v1/conv2/BatchNorm/beta
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_6/bottleneck_v1/conv1/weights:0 from checkpoint resnet_v1_50.ckpt wit

INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_6/bottleneck_v1/conv3/BatchNorm/moving_variance:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_6/bottleneck_v1/conv3/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_1/bottleneck_v1/conv2/BatchNorm/moving_variance:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/unit_1/bottleneck_v1/conv2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable resnet_v1_50/block3/unit_1/bottleneck_v1/conv2/BatchNorm/moving_mean:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block3/unit_1/bottleneck_v1/conv2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_2/bottleneck_v1/conv3/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/unit_2/bottleneck_v1/conv3/weights
INFO:tensorflow:Initialize variable resnet_v1_50/block2/unit_3/bottleneck_v1/conv3/weights:0 from checkpoint resnet_v1_50.ckpt with resnet_v1_50/block2/

INFO:tensorflow:global_step/sec: 2.40393
INFO:tensorflow:loss = 0.14694974, step = 3201 (41.599 sec)
INFO:tensorflow:global_step/sec: 2.40649
INFO:tensorflow:loss = 0.12387383, step = 3301 (41.555 sec)
INFO:tensorflow:global_step/sec: 2.41816
INFO:tensorflow:loss = 0.12761663, step = 3401 (41.354 sec)
INFO:tensorflow:global_step/sec: 2.41241
INFO:tensorflow:loss = 0.1212805, step = 3501 (41.453 sec)
INFO:tensorflow:global_step/sec: 2.41443
INFO:tensorflow:loss = 0.13194332, step = 3601 (41.417 sec)
INFO:tensorflow:global_step/sec: 2.3927
INFO:tensorflow:loss = 0.14245635, step = 3701 (41.795 sec)
INFO:tensorflow:global_step/sec: 2.3829
INFO:tensorflow:loss = 0.15466698, step = 3801 (41.964 sec)
INFO:tensorflow:global_step/sec: 2.39695
INFO:tensorflow:loss = 0.14068937, step = 3901 (41.720 sec)
INFO:tensorflow:global_step/sec: 2.38597
INFO:tensorflow:loss = 0.14200671, step = 4001 (41.912 sec)
INFO:tensorflow:global_step/sec: 2.40734
INFO:tensorflow:loss = 0.15384819, step = 4101 (41.53

In [18]:
%%time
predictions = list(nn.predict(test_input_fn))

INFO:tensorflow:Restoring parameters from /tmp/tmpjwebku45/model.ckpt-6160
CPU times: user 21min 57s, sys: 12.8 s, total: 22min 9s
Wall time: 2min 2s


In [19]:
np.array(predictions).shape
y_truth = test_dataset.labels
y_guess = np.array(predictions)
print("Test AUC: {0:.4f}".format(compute_roc_auc(y_truth, y_guess, CLASSES))) 

Full AUC [0.7889812749842573, 0.8590296734722328, 0.7607959232078535, 0.8805789172123603, 0.8641257317950977, 0.8545835104345408, 0.7613864146649918, 0.8573969366356082, 0.6837787662329343, 0.778462788796783, 0.7023889972324879, 0.7618695878649314, 0.7320212135686417, 0.8158625489955312]
Test AUC: 0.7929
